# 爬取虎扑的赛后评论，并分析

In [1]:
import requests
import re
from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import random
import json

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib.font_manager import _rebuild
_rebuild()

## 找到赛后链接

In [2]:
def order_win_lose(title):
    pattern = '\[赛后\](.*?)(\d+)-(\d+)(.*?)，'
    game_result = re.findall(pattern,title,re.S)
    if len(game_result)>0:
        game_result = game_result[0]
        team_1 = game_result[0]
        team_2 = game_result[3].replace('不敌','')
        if float(game_result[1])>float(game_result[2]):
            return (team_1,team_2)
        else:
            return (team_2,team_1)

In [3]:
order_win_lose('[赛后]公牛86-118不敌火箭，马尔卡宁22+4，戈登31分，保罗13+10，火箭十连胜')

('火箭', '公牛')

In [4]:
# url:'https://bbs.hupu.com/vote'
# sample target: ('https://bbs.hupu.com/21797416.html',
# '[赛后]公牛86-118火箭，马尔卡宁22+4，戈登31分，保罗13+10，火箭十连胜',
# '20')
def get_after_game_post(browser,url):
    browser.get(url)
    page = browser.page_source
    soup = BeautifulSoup(page,'lxml')
    lis = soup.find_all('li')
    targets = []
    for li in lis:
        target = li.find('a',class_='truetit')
        if target!=None and target.text.find('[赛后]')!=-1:
            page_n = li.find('span',class_='multipage').find_all('a')[-1].text 
            targets.append(('https://bbs.hupu.com'+target['href'],order_win_lose(target.text),page_n))
    return targets

## 统计评论人主队

In [5]:
def counter_fans(post_url,page_n):
    counter = Counter()
    i=1
    while i<=page_n:
        try:
            after_game_url ='{}-{}.html'.format(post_url.replace('.html',''),i)
            print('in {}...'.format(after_game_url))
            response = requests.get(after_game_url)
            # <div uname="疯狂的麦克斯4" uid="278913245705307" class="j_u">
            #<a href="https://my.hupu.com/278913245705307" target="_blank" class="headpic"><img src="https://i3.hoopchina.com.cn/user/header/20170415/149222410134740_small_1.jpg" width="45" height="45" alt="疯狂的麦克斯4"></a>
            #</div>
            results = re.findall('<a\sclass="u"\starget="_blank"\shref="https://my.hupu.com/(.*?)".*?>',response.text,re.S)
            fans = []
            for result in results:
                usernum = result
                url = 'https://my.hupu.com/{}'.format(usernum)
                resp = requests.get(url)
                pat = re.compile('<span.*?>NBA主队.*?<a.*?>(.*?)</a>.*?',re.S)
                team = re.findall(pat,resp.text)
                if len(team)>0:
                    fans.append(team[0])
                else:
                    fans.append('无主队')
            counter.update(fans)
            i+=1
        except Exception:
            print('end')
    return counter

In [6]:
posts = []
browser = webdriver.Chrome()
with open('cookies.json') as f:
    cookies = json.load(f)
browser.get('https://bbs.hupu.com/vote')
for c in cookies:
    browser.add_cookie(c)
for i in range(1,700):
    try:
        url = 'https://bbs.hupu.com/vote-{}'.format(i)
        posts.extend(get_after_game_post(browser,url))
        print('In for_loop_{}...'.format(i))
        time.sleep(5*random.random())
    except Exception as e:
        print(e)
browser.close()

In for_loop_1...
In for_loop_2...
In for_loop_3...
In for_loop_4...
In for_loop_5...
In for_loop_6...
In for_loop_7...
In for_loop_8...
In for_loop_9...
In for_loop_10...
In for_loop_11...
In for_loop_12...
In for_loop_13...
In for_loop_14...
In for_loop_15...
In for_loop_16...
In for_loop_17...
In for_loop_18...
In for_loop_19...
In for_loop_20...
In for_loop_21...
In for_loop_22...
In for_loop_23...
In for_loop_24...
In for_loop_25...
In for_loop_26...
In for_loop_27...
In for_loop_28...
In for_loop_29...
In for_loop_30...
In for_loop_31...
In for_loop_32...
In for_loop_33...
In for_loop_34...
In for_loop_35...
In for_loop_36...
In for_loop_37...
In for_loop_38...
In for_loop_39...
In for_loop_40...
In for_loop_41...
In for_loop_42...
In for_loop_43...
In for_loop_44...
In for_loop_45...
In for_loop_46...
In for_loop_47...
In for_loop_48...
In for_loop_49...
In for_loop_50...
In for_loop_51...
In for_loop_52...
In for_loop_53...
In for_loop_54...
In for_loop_55...
In for_loop_56...
I

In for_loop_436...
In for_loop_437...
In for_loop_438...
In for_loop_439...
In for_loop_440...
In for_loop_441...
In for_loop_442...
In for_loop_443...
In for_loop_444...
In for_loop_445...
In for_loop_446...
In for_loop_447...
In for_loop_448...
In for_loop_449...
In for_loop_450...
In for_loop_451...
In for_loop_452...
In for_loop_453...
In for_loop_454...
In for_loop_455...
In for_loop_456...
In for_loop_457...
In for_loop_458...
In for_loop_459...
In for_loop_460...
In for_loop_461...
In for_loop_462...
In for_loop_463...
In for_loop_464...
In for_loop_465...
In for_loop_466...
In for_loop_467...
In for_loop_468...
In for_loop_469...
In for_loop_470...
In for_loop_471...
In for_loop_472...
In for_loop_473...
In for_loop_474...
In for_loop_475...
In for_loop_476...
In for_loop_477...
In for_loop_478...
In for_loop_479...
In for_loop_480...
In for_loop_481...
In for_loop_482...
In for_loop_483...
In for_loop_484...
In for_loop_485...
In for_loop_486...
In for_loop_487...
In for_loop_

In [7]:
posts

[('https://bbs.hupu.com/21877715.html', ('骑士', '奇才'), '50'),
 ('https://bbs.hupu.com/21879043.html', ('掘金', '森林狼'), '33'),
 ('https://bbs.hupu.com/21877570.html', ('火箭', '开拓者'), '38'),
 ('https://bbs.hupu.com/21877050.html', ('步行者', '勇士'), '17'),
 ('https://bbs.hupu.com/21877527.html', ('篮网', '雄鹿'), '6'),
 ('https://bbs.hupu.com/21878330.html', ('爵士', '快船'), '7'),
 ('https://bbs.hupu.com/21842862.html', ('掘金', '雄鹿'), '19'),
 ('https://bbs.hupu.com/21652353.html', ('森林狼', '勇士'), '31'),
 ('https://bbs.hupu.com/21861510.html', ('公牛', '黄蜂'), '5'),
 ('https://bbs.hupu.com/21871203.html', ('湖人', '马刺'), '36'),
 ('https://bbs.hupu.com/21869947.html', ('鹈鹕', '灰熊'), '4'),
 ('https://bbs.hupu.com/21840948.html', ('马刺', '火箭'), '62'),
 ('https://bbs.hupu.com/21861768.html', ('勇士', '雷霆'), '56'),
 ('https://bbs.hupu.com/21861485.html', ('火箭', '奇才'), '19'),
 ('https://bbs.hupu.com/21869973.html', ('猛龙', '凯尔特人'), '9'),
 ('https://bbs.hupu.com/21869575.html', ('76人', '活塞'), '9'),
 ('https://bbs.hupu.com

In [8]:
results=[]
for post in posts:
    if post[1]!=None:
        post_url = post[0]
        page_n = int(post[2])
        win = post[1][0]
        loss = post[1][1]
        counter = counter_fans(post_url,page_n)
        results.append((win,loss,counter))

in https://bbs.hupu.com/21877715-1.html...
in https://bbs.hupu.com/21877715-2.html...
in https://bbs.hupu.com/21877715-3.html...
in https://bbs.hupu.com/21877715-4.html...
in https://bbs.hupu.com/21877715-5.html...
in https://bbs.hupu.com/21877715-6.html...
in https://bbs.hupu.com/21877715-7.html...
in https://bbs.hupu.com/21877715-8.html...
in https://bbs.hupu.com/21877715-9.html...
in https://bbs.hupu.com/21877715-10.html...
in https://bbs.hupu.com/21877715-11.html...
in https://bbs.hupu.com/21877715-12.html...
in https://bbs.hupu.com/21877715-13.html...
in https://bbs.hupu.com/21877715-14.html...
in https://bbs.hupu.com/21877715-15.html...
in https://bbs.hupu.com/21877715-16.html...
in https://bbs.hupu.com/21877715-17.html...
in https://bbs.hupu.com/21877715-18.html...
in https://bbs.hupu.com/21877715-19.html...
in https://bbs.hupu.com/21877715-20.html...
in https://bbs.hupu.com/21877715-21.html...
in https://bbs.hupu.com/21877715-22.html...
in https://bbs.hupu.com/21877715-23.html.

in https://bbs.hupu.com/21652353-19.html...
in https://bbs.hupu.com/21652353-20.html...
in https://bbs.hupu.com/21652353-21.html...
in https://bbs.hupu.com/21652353-22.html...
in https://bbs.hupu.com/21652353-23.html...
in https://bbs.hupu.com/21652353-24.html...
in https://bbs.hupu.com/21652353-25.html...
in https://bbs.hupu.com/21652353-26.html...
in https://bbs.hupu.com/21652353-27.html...
in https://bbs.hupu.com/21652353-28.html...
in https://bbs.hupu.com/21652353-29.html...
in https://bbs.hupu.com/21652353-30.html...
in https://bbs.hupu.com/21652353-31.html...
in https://bbs.hupu.com/21861510-1.html...
in https://bbs.hupu.com/21861510-2.html...
in https://bbs.hupu.com/21861510-3.html...
in https://bbs.hupu.com/21861510-4.html...
in https://bbs.hupu.com/21861510-5.html...
in https://bbs.hupu.com/21871203-1.html...
in https://bbs.hupu.com/21871203-2.html...
in https://bbs.hupu.com/21871203-3.html...
in https://bbs.hupu.com/21871203-4.html...
in https://bbs.hupu.com/21871203-5.html..

in https://bbs.hupu.com/21861485-13.html...
in https://bbs.hupu.com/21861485-14.html...
in https://bbs.hupu.com/21861485-15.html...
in https://bbs.hupu.com/21861485-16.html...
in https://bbs.hupu.com/21861485-17.html...
in https://bbs.hupu.com/21861485-18.html...
in https://bbs.hupu.com/21861485-19.html...
in https://bbs.hupu.com/21869973-1.html...
in https://bbs.hupu.com/21869973-2.html...
in https://bbs.hupu.com/21869973-3.html...
in https://bbs.hupu.com/21869973-4.html...
in https://bbs.hupu.com/21869973-5.html...
in https://bbs.hupu.com/21869973-6.html...
in https://bbs.hupu.com/21869973-7.html...
in https://bbs.hupu.com/21869973-8.html...
in https://bbs.hupu.com/21869973-9.html...
in https://bbs.hupu.com/21869575-1.html...
in https://bbs.hupu.com/21869575-2.html...
in https://bbs.hupu.com/21869575-3.html...
in https://bbs.hupu.com/21869575-4.html...
in https://bbs.hupu.com/21869575-5.html...
in https://bbs.hupu.com/21869575-6.html...
in https://bbs.hupu.com/21869575-7.html...
in h

in https://bbs.hupu.com/21824492-9.html...
in https://bbs.hupu.com/21824492-10.html...
in https://bbs.hupu.com/21824492-11.html...
in https://bbs.hupu.com/21824492-12.html...
in https://bbs.hupu.com/21824492-13.html...
in https://bbs.hupu.com/21824492-14.html...
in https://bbs.hupu.com/21824492-15.html...
in https://bbs.hupu.com/21824492-16.html...
in https://bbs.hupu.com/21824492-17.html...
in https://bbs.hupu.com/21824492-18.html...
in https://bbs.hupu.com/21824492-19.html...
in https://bbs.hupu.com/21824492-20.html...
in https://bbs.hupu.com/21824492-21.html...
in https://bbs.hupu.com/21824492-22.html...
in https://bbs.hupu.com/21824492-23.html...
in https://bbs.hupu.com/21824492-24.html...
in https://bbs.hupu.com/21824492-25.html...
in https://bbs.hupu.com/21824492-26.html...
in https://bbs.hupu.com/21824492-27.html...
in https://bbs.hupu.com/21824492-28.html...
in https://bbs.hupu.com/21824492-29.html...
in https://bbs.hupu.com/21824492-30.html...
in https://bbs.hupu.com/21824492-

in https://bbs.hupu.com/21840778-7.html...
in https://bbs.hupu.com/21841361-1.html...
in https://bbs.hupu.com/21841361-2.html...
in https://bbs.hupu.com/21841361-3.html...
in https://bbs.hupu.com/21841252-1.html...
in https://bbs.hupu.com/21841252-2.html...
in https://bbs.hupu.com/21832164-1.html...
in https://bbs.hupu.com/21832164-2.html...
in https://bbs.hupu.com/21832164-3.html...
in https://bbs.hupu.com/21832164-4.html...
in https://bbs.hupu.com/21833087-1.html...
in https://bbs.hupu.com/21833087-2.html...
in https://bbs.hupu.com/21833087-3.html...
in https://bbs.hupu.com/21833087-4.html...
in https://bbs.hupu.com/21833087-5.html...
in https://bbs.hupu.com/21833087-6.html...
in https://bbs.hupu.com/21833087-7.html...
in https://bbs.hupu.com/21833087-8.html...
in https://bbs.hupu.com/21833087-9.html...
in https://bbs.hupu.com/21833087-10.html...
in https://bbs.hupu.com/21833087-11.html...
in https://bbs.hupu.com/21833087-12.html...
in https://bbs.hupu.com/21833087-13.html...
in http

...

in https://bbs.hupu.com/20565064-26.html...
in https://bbs.hupu.com/20565064-27.html...
in https://bbs.hupu.com/20565064-28.html...
in https://bbs.hupu.com/20565064-29.html...
in https://bbs.hupu.com/20565064-30.html...
in https://bbs.hupu.com/20565064-31.html...
in https://bbs.hupu.com/20570989-1.html...
in https://bbs.hupu.com/20570989-2.html...
in https://bbs.hupu.com/20570989-3.html...
in https://bbs.hupu.com/20570989-4.html...
in https://bbs.hupu.com/20570989-5.html...
in https://bbs.hupu.com/20570989-6.html...
in https://bbs.hupu.com/20570989-7.html...
in https://bbs.hupu.com/20570989-8.html...
in https://bbs.hupu.com/20570989-9.html...
in https://bbs.hupu.com/20570989-10.html...
in https://bbs.hupu.com/20570989-11.html...
in https://bbs.hupu.com/20570989-12.html...
in https://bbs.hupu.com/20570989-13.html...
in https://bbs.hupu.com/20570989-14.html...
in https://bbs.hupu.com/20570989-15.html...
in https://bbs.hupu.com/20570989-16.html...
in https://bbs.hupu.com/20570989-17.html.

in https://bbs.hupu.com/20550350-42.html...
in https://bbs.hupu.com/20550350-43.html...
in https://bbs.hupu.com/20550350-44.html...
in https://bbs.hupu.com/20550350-45.html...
in https://bbs.hupu.com/20550350-46.html...
in https://bbs.hupu.com/20550350-47.html...
in https://bbs.hupu.com/20550350-48.html...
in https://bbs.hupu.com/20550350-49.html...
in https://bbs.hupu.com/20550350-50.html...
in https://bbs.hupu.com/20550350-51.html...
in https://bbs.hupu.com/20550350-52.html...
in https://bbs.hupu.com/20550350-53.html...
in https://bbs.hupu.com/20550350-54.html...
in https://bbs.hupu.com/20550350-55.html...
in https://bbs.hupu.com/20550350-56.html...
in https://bbs.hupu.com/20550350-57.html...
in https://bbs.hupu.com/20550350-58.html...
in https://bbs.hupu.com/20550350-59.html...
in https://bbs.hupu.com/20550350-60.html...
in https://bbs.hupu.com/20550350-61.html...
in https://bbs.hupu.com/20550350-62.html...
in https://bbs.hupu.com/20550350-63.html...
in https://bbs.hupu.com/20550350

in https://bbs.hupu.com/20550654-4.html...
in https://bbs.hupu.com/20550654-5.html...
in https://bbs.hupu.com/20550654-6.html...
in https://bbs.hupu.com/20550654-7.html...
in https://bbs.hupu.com/20550654-8.html...
in https://bbs.hupu.com/20550654-9.html...
in https://bbs.hupu.com/20550654-10.html...
in https://bbs.hupu.com/20550654-11.html...
in https://bbs.hupu.com/20550654-12.html...
in https://bbs.hupu.com/20550654-13.html...
in https://bbs.hupu.com/20550654-14.html...
in https://bbs.hupu.com/20550654-15.html...
in https://bbs.hupu.com/20550654-16.html...
in https://bbs.hupu.com/20550654-17.html...
in https://bbs.hupu.com/20550654-18.html...
in https://bbs.hupu.com/20550654-19.html...
in https://bbs.hupu.com/20550654-20.html...
in https://bbs.hupu.com/20550654-21.html...
in https://bbs.hupu.com/20550654-22.html...
in https://bbs.hupu.com/20550654-23.html...
in https://bbs.hupu.com/20550654-24.html...
in https://bbs.hupu.com/20550654-25.html...
in https://bbs.hupu.com/20550654-26.ht

in https://bbs.hupu.com/20528487-7.html...
in https://bbs.hupu.com/20528487-8.html...
in https://bbs.hupu.com/20528487-9.html...
in https://bbs.hupu.com/20528487-10.html...
in https://bbs.hupu.com/20528487-11.html...
in https://bbs.hupu.com/20528487-12.html...
in https://bbs.hupu.com/20528487-13.html...
in https://bbs.hupu.com/20528487-14.html...
in https://bbs.hupu.com/20528487-15.html...
in https://bbs.hupu.com/20528487-16.html...
in https://bbs.hupu.com/20528487-17.html...
in https://bbs.hupu.com/20528487-18.html...
in https://bbs.hupu.com/20528487-19.html...
in https://bbs.hupu.com/20528487-20.html...
in https://bbs.hupu.com/20528487-21.html...
in https://bbs.hupu.com/20528487-22.html...
in https://bbs.hupu.com/20528487-23.html...
in https://bbs.hupu.com/20528487-24.html...
in https://bbs.hupu.com/20528487-25.html...
in https://bbs.hupu.com/20537630-1.html...
in https://bbs.hupu.com/20537630-2.html...
in https://bbs.hupu.com/20537630-3.html...
in https://bbs.hupu.com/20537630-4.htm

in https://bbs.hupu.com/20522505-82.html...
in https://bbs.hupu.com/20522505-83.html...
in https://bbs.hupu.com/20522505-84.html...
in https://bbs.hupu.com/20522505-85.html...
in https://bbs.hupu.com/20522505-86.html...
in https://bbs.hupu.com/20522505-87.html...
in https://bbs.hupu.com/20522505-88.html...
in https://bbs.hupu.com/20522505-89.html...
in https://bbs.hupu.com/20522505-90.html...
in https://bbs.hupu.com/20522505-91.html...
in https://bbs.hupu.com/20522505-92.html...
in https://bbs.hupu.com/20522505-93.html...
in https://bbs.hupu.com/20522505-94.html...
in https://bbs.hupu.com/20522505-95.html...
in https://bbs.hupu.com/20522505-96.html...
in https://bbs.hupu.com/20522505-97.html...
in https://bbs.hupu.com/20522505-98.html...
in https://bbs.hupu.com/20522505-99.html...
in https://bbs.hupu.com/20522505-100.html...
in https://bbs.hupu.com/20522505-101.html...
in https://bbs.hupu.com/20522505-102.html...
in https://bbs.hupu.com/20522505-103.html...
in https://bbs.hupu.com/2052

In [9]:
result_df = pd.DataFrame(results)
result_df.columns = ['win','loss','count']

In [10]:
print(len(result_df))
result_df.head()

1172


,win,loss,count
0,骑士,奇才,"{'火箭': 43, '无主队': 903, '黄蜂': 90, '湖人': 37, '骑士..."
1,掘金,森林狼,"{'火箭': 61, '无主队': 612, '马刺': 54, '雷霆': 21, '骑士..."
2,火箭,开拓者,"{'火箭': 160, '无主队': 667, '凯尔特人': 2, '湖人': 11, '..."
3,步行者,勇士,"{'火箭': 37, '骑士': 21, '无主队': 390, '雷霆': 12, '勇士..."
4,篮网,雄鹿,"{'火箭': 14, '无主队': 105, '篮网': 7, '雄鹿': 4, '湖人':..."


In [11]:
count_table = pd.DataFrame()
for i in range(len(result_df)):
    s1 = pd.Series(result_df['count'][i])
    count_table = pd.concat([count_table,s1],axis=1)

In [12]:
count_table = count_table.T
count_table.reset_index(inplace=True)

In [13]:
final_table = pd.concat([result_df[['win','loss']],count_table],axis=1)
final_table = final_table.drop('index',axis=1)
final_table = final_table.fillna(0)

In [14]:
final_table

,win,loss,76人,公牛,凯尔特人,勇士,国王,太阳,奇才,尼克斯,...,猛龙,篮网,老鹰,雄鹿,雷霆,马刺,骑士,魔术,鹈鹕,黄蜂
0,骑士,奇才,3.0,1.0,4.0,4.0,0.0,0.0,13.0,0.0,...,2.0,1.0,0.0,0.0,1.0,4.0,135.0,0.0,0.0,90.0
1,掘金,森林狼,0.0,1.0,33.0,26.0,0.0,1.0,1.0,10.0,...,8.0,1.0,0.0,2.0,21.0,54.0,46.0,0.0,0.0,0.0
2,火箭,开拓者,1.0,0.0,2.0,3.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,2.0,14.0,11.0,0.0,0.0,2.0
3,步行者,勇士,2.0,2.0,1.0,32.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,1.0,12.0,0.0,21.0,0.0,1.0,0.0
4,篮网,雄鹿,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,4.0,0.0,1.0,3.0,0.0,0.0,0.0
5,爵士,快船,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
6,掘金,雄鹿,2.0,0.0,7.0,5.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,8.0,31.0,13.0,1.0,0.0,0.0,0.0
7,森林狼,勇士,0.0,1.0,3.0,48.0,0.0,0.0,0.0,4.0,...,1.0,0.0,3.0,8.0,15.0,16.0,20.0,0.0,0.0,0.0
8,公牛,黄蜂,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,36.0
9,湖人,马刺,0.0,2.0,2.0,18.0,0.0,8.0,0.0,1.0,...,0.0,0.0,0.0,0.0,8.0,64.0,13.0,0.0,0.0,2.0


In [16]:
with open('result_table.csv','w') as file:
    final_table.to_csv(file)

In [30]:
len(final_table)

1172

### 分析结果
- 在湿乎乎评论的球迷分布
- 屠龟大会的球迷分布
- 雷霆赢球的球迷分布

In [17]:
from pyecharts import Bar
fans = final_table.iloc[:,2:].sum().drop('无主队').sort_values(ascending=False)
attr = list(fans.index)
bar = Bar('球迷数量')
bar.add('fans_count',attr,fans,xaxis_rotate=90)
bar

In [18]:
def plot_bar(team):
    win = final_table[final_table['win']==team]
    loss = final_table[final_table['loss']==team]
    win_counts = win.iloc[:,2:].mean().drop('无主队')
    loss_counts = loss.iloc[:,2:].mean().drop('无主队')
    attr1 = list(win_counts.index)
    attr2 = list(loss_counts.index)
    bar = Bar("{}比赛球迷分布--win:{}--loss:{}".format(team,len(win),len(loss)))
    bar.add("win", attr1,win_counts,xaxis_rotate=90)
    bar.add('loss',attr2,loss_counts,xaxis_rotate=90)
    return bar

In [19]:
thunder_bar = plot_bar('雷霆')

In [20]:
thunder_bar

In [21]:
warrior_bar = plot_bar('勇士')
warrior_bar

In [22]:
cav_bar = plot_bar('骑士')
cav_bar

In [23]:
spurs_bar = plot_bar('马刺')
spurs_bar

In [24]:
plot_bar('火箭')

In [25]:
plot_bar('湖人')

In [26]:
plot_bar('快船')

In [27]:
plot_bar('76人')

In [28]:
plot_bar('凯尔特人')

In [29]:
plot_bar('森林狼')